In [18]:
from extract_grammar import GrammaticalQuadrupleExtraction
from data.grammar import grammar_str7
from eliminate_left_recursion import auto_eliminate_left_recursion
from print_productions import print_productions

In [194]:
from data.grammar import grammar_str2

# 示例用法
extractor = GrammaticalQuadrupleExtraction()
grammar_string2 = "Ems ::= Ems + Tfs | Tfs\nTfs ::= Tfs * F' | F'\nF' ::= ( Ems ) | iss\n\n\n\n\n"
grammar_string = grammar_str2

non_terminators, terminators, production, start = extractor.extract_grammar_components(grammar_string)
print("terminators: ",terminators)
print("non_terminators: ",non_terminators)
print("production: ",production)
print("start: ",start)

In [43]:
from eliminate_left_recursion import add_production
import copy


def remove_production(non_terminal, production, productions):
    if non_terminal in productions and production in productions[non_terminal]:
        productions[non_terminal].remove(production)
        if not productions[non_terminal]:  # If the set becomes empty, remove the key
            del productions[non_terminal]

def eliminate_rules(key_item,value_item,production):
    end_production = copy.deepcopy(production)
    mid_key_item = key_item
    end_key_item = mid_key_item + "'"
    mid_value_item = value_item.copy()
    end_value_item = value_item.copy()
    # 先改后删
    for i in range(len(end_value_item)):
        if end_value_item[i] == mid_key_item:
            end_value_item.pop(i)
            end_value_item.append(end_key_item)
            break
    for item in end_production[mid_key_item]:
        if item == mid_value_item: continue
        if mid_key_item in item: continue
        if end_key_item in item: continue
        item.append(end_key_item)
    remove_production(mid_key_item,mid_value_item,end_production)
    add_production(end_key_item,end_value_item,end_production)
    if ['ε'] not in end_production[end_key_item]:
        add_production(end_key_item,['ε'],end_production)
    else:
        remove_production(end_key_item,['ε'],end_production)
        add_production(end_key_item,['ε'],end_production)
    return end_production
        
def auto_eliminate_rules(productions):
    end_productions = copy.deepcopy(productions)
    tag = True
    while tag:
        tag = False
        for key in end_productions:
            for item in end_productions[key]:
                # 消去规则左递归
                if key in item and len(key) == 1:
                    tag = True
                    end_productions = eliminate_rules(key,item,end_productions)  
    return end_productions


def eliminate_grammar(productions):
    mid_productions = copy.deepcopy(productions)
    end_productions = copy.deepcopy(productions)
    mid_changed_keys_list = []
    for key in productions:
        if "'" in key:
            mid_changed_keys_list.append(key.replace("'",''))
    flg = False
    for key in mid_productions:
        if key in mid_changed_keys_list: continue
        for item in mid_productions[key]:
            tag = False
            for i in range(len(item)):
                if item[i] in mid_changed_keys_list:
                    tag = True
                    flg = True
                    for item_i in mid_productions[item[i]]:
                        end_item = item.copy()
                        end_item.pop(i)
                        end_item[i:i] = item_i
                        add_production(key,end_item,end_productions)
            if tag:
                remove_production(key,item,end_productions)
    return end_productions,flg           

**检验输出**

### 自动化

In [44]:
def auto_eliminate_left_recursion(productions):
    end_productions = copy.deepcopy(productions)
    while True:  
        end_productions = auto_eliminate_rules(end_productions)
        end_productions,flg = eliminate_grammar(end_productions)
        if not flg:
            break
    return end_productions

**检验输出**

In [195]:
print(print_productions(production))
mid_production = auto_eliminate_left_recursion(production)
print("\n"+print_productions(mid_production))


In [250]:
non_terminators = ['E', 'T', 'F', "E'", "T'"]
mid_production = {'E': [['T', "E'"]], 'T': [['F', "T'"]], 'F': [['i'],['(', "E", ")"]], "E'": [['ε'], ['+', "T", "E'"]], "T'": [['ε'], ['*', "F", "T'"]]}

print("terminators: ",terminators)
print("non_terminators: ",non_terminators)
print("production: ",mid_production)
print("start: ",start)

In [251]:
def calculate_first(terminators, non_terminators, production):
    first = {symbol: set() for symbol in non_terminators}

    def calculate_symbol_first(symbol):
        for production_rule in production[symbol]:
            # print("symbol and production_rule: ",symbol,production_rule)
            if production_rule[0] in terminators or production_rule[0] == 'ε':
                first[symbol].add(production_rule[0])
                continue
            elif production_rule[0] in non_terminators:
                mid_production_rule = production[production_rule[0]]
                if mid_production_rule[0] in terminators:
                    first[symbol] |= {mid_production_rule[0]}
                else:
                    for element in production_rule:
                        if element in terminators:
                            first[symbol] |= {element}
                            break
                        if element in non_terminators:
                            mid_first = calculate_symbol_first(element)
                            if 'ε' not in mid_first:
                                first[symbol] |= calculate_symbol_first(element)
                                break
                            first[symbol] |= calculate_symbol_first(element)
                        
        return first[symbol]

    for symbol in non_terminators:
        calculate_symbol_first(symbol)

    return first

In [252]:
first = calculate_first(terminators, non_terminators, mid_production)
print("First 集合:")
for symbol, first_set in first.items():
    print(symbol, ":", first_set)

In [253]:
def calculate_follow(start, terminators, non_terminators, production, first):
    follow = {symbol: set() for symbol in non_terminators}
    follow[start].add('#')

    def calculate_symbol_follow(symbol):
        for head, body in production.items():
            for rule in body:
                for i, element in enumerate(rule):
                    if element == symbol:
                        if i == len(rule)-1:
                            follow[symbol] |= follow[head]
                            break
                        
                        remaining_symbols = rule[i+1:]

                        for next_symbol in remaining_symbols:
                            if next_symbol in terminators:
                                follow[symbol].add(next_symbol)
                                break
                            elif 'ε' not in first[next_symbol]:
                                follow[symbol] |= first[next_symbol]
                                break
                            else:
                                follow[symbol] |= first[next_symbol] - {'ε'}
                                follow[symbol] |= calculate_symbol_follow(head)

        return follow[symbol]

    for symbol in non_terminators:
        calculate_symbol_follow(symbol)

    return follow


In [254]:
follow = calculate_follow(start, terminators, non_terminators, mid_production, first)
print("\nFollow 集合:")
for symbol, follow_set in follow.items():
    print(symbol, ":", follow_set)

In [255]:
def construct_parsing_table(terminators, non_terminators, production, first, follow):
    parsing_table = {non_terminal: {terminator: [] for terminator in terminators + ['#']} for non_terminal in non_terminators}

    for head, bodies in production.items():
        for body in bodies:
            first_set = set()
            if body[0] == 'ε':
                first_set = {'ε'}
            else:
                for symbol in body:
                    if symbol in terminators:
                        first_set.add(symbol)
                        break
                    first_set |= first[symbol]
                    if 'ε' not in first[symbol]:
                        break
                    first_set.discard('ε')
            
            for terminal in first_set:
                if terminal != 'ε':
                    parsing_table[head][terminal].append(body)
            
            if 'ε' in first_set or body[0] == 'ε':
                for terminal in follow[head]:
                    parsing_table[head][terminal].append(body)
                    
    return parsing_table

In [295]:
parsing_table = construct_parsing_table(terminators, non_terminators, mid_production, first, follow)

print("\n预测分析表:")
print("{:<10} {:<10} {:<10}".format("非终结符", "终结符", "产生式"))
for non_terminal, terminals in parsing_table.items():
    for terminal, rules in terminals.items():
        if rules:
            print("{:<12} {:<12} {:<10}".format(str(non_terminal), str(terminal), ''.join(rules[0])))
            # print(non_terminal, ",", terminal, ":", rules)
print(parsing_table)

In [296]:
parsing_table

In [297]:
import pygraphviz as pgv

class TreeNode:
    def __init__(self, symbol):
        self.symbol = symbol
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

class PredictiveParser:
    def __init__(self, table, start):
        self.table = table
        self.start = start
        self.stack = [('#', None), (start, TreeNode(start))]  # Stack now holds tuples of (symbol, TreeNode)
        self.input_str = ""
        self.current_index = 0
        self.root = self.stack[1][1]  # Root of the syntax tree

    def parse(self, input_str):
        self.input_str = input_str + '#'  # Add end marker
        self.current_index = 0
        
        step = 0
        print(f"{'步骤':<10}{'栈':<30}{'输入':<30}{'动作'}")
        print("-" * 80)
        
        while len(self.stack) > 0:
            top, node = self.stack.pop()
            current_char = self.input_str[self.current_index]
            stack_content = ''.join(symbol for symbol, _ in reversed(self.stack))
            stack_content = stack_content[::-1]
            input_content = self.input_str[self.current_index:]
            
            if top == 'ε':
                step += 1
                print(f"{step:<10}{stack_content:<30}{input_content:<30}弹出 ε")
                continue
            elif top in self.table:
                production = self.table[top].get(current_char)
                if production and production != [[]]:
                    step += 1
                    action = f"用 {top} -> {''.join([''.join(p) for p in production])}"
                    print(f"{step:<10}{stack_content:<30}{input_content:<30}{action}")
                    children_nodes = []
                    for symbol in reversed(production[0]):
                        child_node = TreeNode(symbol)
                        children_nodes.append(child_node)
                        self.stack.append((symbol, child_node))
                    node.children.extend(reversed(children_nodes))
                else:
                    print(f"Error: No production found for table[{top}][{current_char}]")
                    return False
            else:
                if top == current_char:
                    step += 1
                    action = f"匹配 {top}"
                    print(f"{step:<10}{stack_content:<30}{input_content:<30}{action}")
                    if node:  # If the node is not None (it would be None for '#')
                        leaf_node = TreeNode(top)
                        node.add_child(leaf_node)
                    if top == '#':
                        return True
                    self.current_index += 1
                else:
                    print(f"Error: Expected {top}, but found {current_char}")
                    return False
        
        if self.current_index == len(self.input_str) - 1:  # Check if all input is consumed
            print("Parsing successful!")
            return True
        else:
            print(f"Error: Input not fully consumed, index at {self.current_index}")
            return False

    def print_syntax_tree(self, node=None, indent=""):
        if node is None:
            node = self.root
        print(indent + node.symbol)
        for child in node.children:
            self.print_syntax_tree(child, indent + "  ")

    def draw_syntax_tree(self, filename="syntax_tree.png"):
        graph = pgv.AGraph(strict=False, directed=True)
        self._add_nodes_edges(graph, self.root)
        graph.layout(prog='dot')
        graph.draw(filename)

    def _add_nodes_edges(self, graph, node, parent_name=None):
        node_name = str(id(node))
        graph.add_node(node_name, label=node.symbol)
        if parent_name:
            graph.add_edge(parent_name, node_name)
        for child in node.children:
            self._add_nodes_edges(graph, child, node_name)

# 调用解析器并绘制语法树的函数
def parse_and_draw(input_str, parsing_table, start_symbol, output_file="syntax_tree.png"):
    parser = PredictiveParser(parsing_table, start_symbol)
    result = parser.parse(input_str)
    print(f"Result: {'Success' if result else 'Failure'}")
    
    # 打印语法树
    parser.print_syntax_tree()
    
    # 绘制语法树
    parser.draw_syntax_tree(output_file)
    print(f"Syntax tree saved to {output_file}")

# 语法分析表


# 调用函数解析字符串并绘制语法树
parse_and_draw("(i+i)*i", parsing_table, start, "syntax_tree.png")


In [ ]:
parsing_table = {
    'E': {
        '(': ['T', 'E\''],
        'i': ['T', 'E\''],
    },
    'E\'': {
        '+': ['+', 'T', 'E\''],
        ')': ['ε'],
        '#': ['ε'],
    },
    'T': {
        '(': ['F', 'T\''],
        'i': ['F', 'T\''],
    },
    'T\'': {
        '+': ['ε'],
        '*': ['*', 'F', 'T\''],
        ')': ['ε'],
        '#': ['ε'],
    },
    'F': {
        '(': ['(', 'E', ')'],
        'i': ['i'],
    }
}

In [273]:
class PredictiveParser:
    def __init__(self, table, start):
        self.table = table
        self.start = start
        self.stack = ['#'] + [start]
        self.input_str = ""
        self.current_index = 0

    def parse(self, input_str):
        self.input_str = input_str + '#'  # 添加结束标志
        self.current_index = 0
        # self.stack = ['#', 'E']
        print(f"Input: {self.input_str}")
        
        step = 0
        print(f"{'步骤':<10}{'栈':<30}{'输入':<30}{'动作'}")
        print("-" * 80)
        
        while len(self.stack) > 0:
            top = self.stack.pop()
            current_char = self.input_str[self.current_index]
            stack_content = ''.join(reversed(self.stack))
            stack_content = stack_content[::-1]
            input_content = self.input_str[self.current_index:]
            
            if top == 'ε':
                step += 1
                print(f"{step:<10}{stack_content:<30}{input_content:<30}弹出 ε")
                continue
            elif top in self.table:
                production = self.table[top][current_char]
                if production:
                    step += 1
                    action = f"用 {top} -> {''.join(production[0])}"
                    print(f"{step:<10}{stack_content:<30}{input_content:<30}{action}")
                    for symbol in reversed(production[0]):
                        self.stack.append(symbol)
                else:
                    print(f"Error: No production found for table[{top}][{current_char}]")
                    return False
            else:
                if top == current_char:
                    step += 1
                    action = f"匹配 {top}"
                    print(f"{step:<10}{stack_content:<30}{input_content:<30}{action}")
                    if top == '#':
                        return True
                    self.current_index += 1
                else:
                    print(f"Error: Expected {top}, but found {current_char}")
                    return False
        
        if self.current_index == len(self.input_str) - 1:  # 检查是否所有输入都被消费
            print("Parsing successful!")
            return True
        else:
            print(f"Error: Input not fully consumed, index at {self.current_index}")
            return False

# 使用预测分析器解析字符串 "(i+i)*i"
parser = PredictiveParser(parsing_table,start)
result = parser.parse("(i+i)*i")
print(f"Result: {'Success' if result else 'Failure'}")

In [292]:
import pygraphviz as pgv

class TreeNode:
    def __init__(self, symbol):
        self.symbol = symbol
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

class PredictiveParser:
    def __init__(self, table, start):
        self.table = table
        self.start = start
        self.stack = [('#', None), (start, TreeNode(start))]  # Stack now holds tuples of (symbol, TreeNode)
        self.input_str = ""
        self.current_index = 0
        self.root = self.stack[1][1]  # Root of the syntax tree

    def parse(self, input_str):
        self.input_str = input_str + '#'  # Add end marker
        self.current_index = 0
        
        step = 0
        print(f"{'步骤':<10}{'栈':<30}{'输入':<30}{'动作'}")
        print("-" * 80)
        
        while len(self.stack) > 0:
            top, node = self.stack.pop()
            current_char = self.input_str[self.current_index]
            stack_content = ''.join(symbol for symbol, _ in reversed(self.stack))
            stack_content = stack_content[::-1]
            input_content = self.input_str[self.current_index:]
            
            if top == 'ε':
                step += 1
                print(f"{step:<10}{stack_content:<30}{input_content:<30}弹出 ε")
                continue
            elif top in self.table:
                production = self.table[top].get(current_char)
                if production:
                    step += 1
                    action = f"用 {top} -> {''.join(production)}"
                    print(f"{step:<10}{stack_content:<30}{input_content:<30}{action}")
                    children_nodes = []
                    for symbol in reversed(production):
                        child_node = TreeNode(symbol)
                        children_nodes.append(child_node)
                        self.stack.append((symbol, child_node))
                    node.children.extend(reversed(children_nodes))
                else:
                    print(f"Error: No production found for table[{top}][{current_char}]")
                    return False
            else:
                if top == current_char:
                    step += 1
                    action = f"匹配 {top}"
                    print(f"{step:<10}{stack_content:<30}{input_content:<30}{action}")
                    if node:  # If the node is not None (it would be None for '#')
                        leaf_node = TreeNode(top)
                        node.add_child(leaf_node)
                    if top == '#':
                        return True
                    self.current_index += 1
                else:
                    print(f"Error: Expected {top}, but found {current_char}")
                    return False
        
        if self.current_index == len(self.input_str) - 1:  # Check if all input is consumed
            print("Parsing successful!")
            return True
        else:
            print(f"Error: Input not fully consumed, index at {self.current_index}")
            return False

    def print_syntax_tree(self, node=None, indent=""):
        if node is None:
            node = self.root
        print(indent + node.symbol)
        for child in node.children:
            self.print_syntax_tree(child, indent + "  ")

    def draw_syntax_tree(self, filename="syntax_tree.png"):
        graph = pgv.AGraph(strict=False, directed=True)
        self._add_nodes_edges(graph, self.root)
        graph.layout(prog='dot')
        graph.draw(filename)

    def _add_nodes_edges(self, graph, node, parent_name=None):
        node_name = str(id(node))
        graph.add_node(node_name, label=node.symbol)
        if parent_name:
            graph.add_edge(parent_name, node_name)
        for child in node.children:
            self._add_nodes_edges(graph, child, node_name)

# 使用预测分析器解析字符串 "(i+i)*i"
# parsing_table = {
#     'E': {
#         '(': ['T', 'E\''],
#         'i': ['T', 'E\''],
#     },
#     'E\'': {
#         '+': ['+', 'T', 'E\''],
#         ')': ['ε'],
#         '#': ['ε'],
#     },
#     'T': {
#         '(': ['F', 'T\''],
#         'i': ['F', 'T\''],
#     },
#     'T\'': {
#         '+': ['ε'],
#         '*': ['*', 'F', 'T\''],
#         ')': ['ε'],
#         '#': ['ε'],
#     },
#     'F': {
#         '(': ['(', 'E', ')'],
#         'i': ['i'],
#     }
# }
start_symbol = 'E'
parser = PredictiveParser(parsing_table, start_symbol)
result = parser.parse("(i+i)*i")
print(f"Result: {'Success' if result else 'Failure'}")

# 打印语法树
# parser.print_syntax_tree()

# 绘制语法树
parser.draw_syntax_tree("syntax_tree.png")
